In [8]:
import pandas as pd
import requests
import matplotlib.pyplot as plt

from keys import Trip_advisor_key
from keys import geoapify_key

In [9]:

# Defining the District dataframe

Districts_df = pd.DataFrame({
    'Location': ['Toronto Distillery District', 'Toronto GTA Area', 'Toronto Downtown', 'Toronto King West', 'Toronto Queen West' , 'Toronto College West', 'Toronto Yorkville','Toronto Old Town', 'Toronto Airport', 'Ottawa City Centre' , 'Niagara Falls'],
    'Lat': [43.6503, 43.6426 , 43.655064 , 43.642567, 43.647044, 43.6552,  43.6707 , 43.3906, 43.6817, 45.4236, 43.087586], 
    'Lng': [-79.3596, -79.3871, -79.386563 , -79.387054, -79.403698, -79.4139,  -79.3930 , -79.2253 ,  -79.6145, -75.7009, -79.081206]  
})

Districts_df.to_csv("output_data/Districts.csv", index_label="Restaurant ID")

# Defining the Restaurant DataFrame

Restaurants_df = pd.DataFrame(columns=['Location', 'Restaurant Name', 'Postal Code', 'Restaurant Address', 'Lat', 'Lng'])

# Loop for districts
err = 0

radius = 1000

params = {
    'categories': 'catering.restaurant',
    'apiKey': geoapify_key,
    'limit':50
}

base_url = 'https://api.geoapify.com/v2/places'

for index, row in Districts_df.iterrows():
    latitude = row['Lat']
    longitude = row['Lng']

    params['filter'] = f'circle:{longitude},{latitude},{radius}'
    params['bias'] = f'proximity:{longitude},{latitude}'

    response = requests.get(base_url, params=params).json()
    for restaurant in response.get('features', []):
        try:
            
            new_row = {
                            'Location': row['Location'],
                            'Restaurant Name': restaurant["properties"]["address_line1"],
                            'Postal Code' : restaurant["properties"]["postcode"],
                            'Restaurant Address': restaurant["properties"]["address_line2"],
                            'Lat': restaurant["geometry"]["coordinates"][1],
                            'Lng': restaurant["geometry"]["coordinates"][0]
                        }
            
            Restaurants_df.loc[len(Restaurants_df)] = new_row
        except(KeyError, IndexError):
            err = err + 1


Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.


In [10]:
Restaurants_df.head()

,Location,Restaurant Name,Postal Code,Restaurant Address,Lat,Lng
0,Toronto Distillery District,Archeo Trattoria,M5A 3C4,"31 Trinity Street, Old Toronto, ON M5A 3C4, Ca...",43.650682,-79.359630
1,Toronto Distillery District,El Catrin,M5A 3C4,"Mill Street, Old Toronto, ON M5A 3C4, Canada",43.650693,-79.359245
2,Toronto Distillery District,Oyster House & grill,M5A 3C4,"Tank House Lane, Old Toronto, ON M5A 3C4, Canada",43.650350,-79.358811
3,Toronto Distillery District,Pure Spirits,M5A 3C4,"17 Tank House Lane, Old Toronto, ON M5A 3C4, C...",43.650395,-79.358754
4,Toronto Distillery District,The Fermenting Cellar,M5A 3C4,"28 Distillery Lane, Old Toronto, ON M5A 3C4, C...",43.649806,-79.360538


In [11]:
#Drill for location_id

Restaurants_df['Location ID'] = ''

err_count = 0


for index, restaurant in Restaurants_df.iterrows():
    
    url = f"https://api.content.tripadvisor.com/api/v1/location/search?key={Trip_advisor_key}&searchQuery={restaurant['Restaurant Name']}&language=en"

    headers = {"accept": "application/json"}

    response = requests.get(url, headers=headers).json()
    
    try:
        
        for res in response['data']:
    
            if res['address_obj']['postalcode'] == restaurant['Postal Code']:
            
                Restaurants_df.at[index, 'Location ID'] = res['location_id']
                
                break
    
    except Exception as e:
        
        err_count = err_count +1
        
        

print(err_count)
Restaurants_df.head()

182


,Location,Restaurant Name,Postal Code,Restaurant Address,Lat,Lng,Location ID
0,Toronto Distillery District,Archeo Trattoria,M5A 3C4,"31 Trinity Street, Old Toronto, ON M5A 3C4, Ca...",43.650682,-79.359630,5078107
1,Toronto Distillery District,El Catrin,M5A 3C4,"Mill Street, Old Toronto, ON M5A 3C4, Canada",43.650693,-79.359245,4556927
2,Toronto Distillery District,Oyster House & grill,M5A 3C4,"Tank House Lane, Old Toronto, ON M5A 3C4, Canada",43.650350,-79.358811,
3,Toronto Distillery District,Pure Spirits,M5A 3C4,"17 Tank House Lane, Old Toronto, ON M5A 3C4, C...",43.650395,-79.358754,708144
4,Toronto Distillery District,The Fermenting Cellar,M5A 3C4,"28 Distillery Lane, Old Toronto, ON M5A 3C4, C...",43.649806,-79.360538,


In [12]:
#Drop row with empty records
Restaurants_df = Restaurants_df[Restaurants_df['Location ID'] != '']

In [13]:
# Export the City_Data into a csv
Restaurants_df.to_csv("output_data/restaurants.csv", index_label="Restaurant ID")

In [14]:
Restaurants_df

,Location,Restaurant Name,Postal Code,Restaurant Address,Lat,Lng,Location ID
0,Toronto Distillery District,Archeo Trattoria,M5A 3C4,"31 Trinity Street, Old Toronto, ON M5A 3C4, Ca...",43.650682,-79.359630,5078107
1,Toronto Distillery District,El Catrin,M5A 3C4,"Mill Street, Old Toronto, ON M5A 3C4, Canada",43.650693,-79.359245,4556927
3,Toronto Distillery District,Pure Spirits,M5A 3C4,"17 Tank House Lane, Old Toronto, ON M5A 3C4, C...",43.650395,-79.358754,708144
5,Toronto Distillery District,Cluny,M5A 3C4,"Tank House Lane, Old Toronto, ON M5A 3C4, Canada",43.650604,-79.357865,6762792
20,Toronto Distillery District,Fusilli Ristorante,M5A 1V1,"531 Queen Street East, Old Toronto, ON M5A 1V1...",43.657075,-79.357150,704440
...,...,...,...,...,...,...,...
419,Niagara Falls,Antica Pizzeria & Ristorante,L2G 3L6,"5785 Victoria Avenue, Niagara Falls, ON L2G 3L...",43.091528,-79.077390,767950
420,Niagara Falls,Falls Deli,L2G 7X5,"6380 Fallsview Boulevard, Niagara Falls, ON L2...",43.082845,-79.079611,11681114
422,Niagara Falls,Brasa,L2G 3V9,"6361 Fallsview Boulevard, Niagara Falls, ON L2...",43.082453,-79.083077,1546799
425,Niagara Falls,Spicy Olive Bar + Grille,L2G 3N7,"Centre Street, Niagara Falls, ON L2G 3N7, Canada",43.092353,-79.076952,5921041
